In [1]:
import gensim
import pandas as pd
from scipy import spatial
import numpy as np
import json
import os



class args(object):

    
    embedding = '../src/WKN-vectors/WKN-vectors.bin'
    ranklib_output = 'w2vwebisall'
    score_output = 'w2vwebisall.txt'
    embedding_type = 'wikipedia2vec'
    
    
    #linker = "Data/geeer_ready.csv"
    linker = '../webis/webis-qinc-22.json'
    dbpedia_input = "../src/DBpedia-Entity/runs/v2/bm25f-ca_v2.run"
    def __init__(self, i_var):
        self.i_var = i_var

path_to_dbpedia = "../src/DBpedia-Entity"

In [2]:
def entity_converter(word, reverse = False, nospace = True):
    # Input: an entity string in dbpedia format
    # Output: string in wikipedia2vec format or readible format
    # Getting entities in the right format
    if args.embedding_type == 'rdf2vec':
            word = word.replace("<dbpedia:", "http://dbpedia.org/resource/")
            word = word.replace(">", "")
            return(word)
    elif args.embedding_type == 'complex':
        word = word.replace("dbpedia:", "http://dbpedia.org/resource/")
        return(word)
    elif reverse:
        word = word.replace("<dbpedia:", "")
        word = word.replace(">", "")
        if nospace:
            return word
        else:
            word = word.replace("_", " ")
            return word
    else:
        word = word.replace("<dbpedia:", "ENTITY/")
        word = word.replace(">", "")
        return word

In [3]:
def entity_lookup(tag, model, tagme = False):
    # Input: entity string
    # Output: True wikipedia2vec format
    # Looking up the embedding of entities, returning [] when entity not in corpus
    if tagme and args.embedding_type == 'rdf2vec':
        tag = tag.replace('ENTITY/', 'http://dbpedia.org/resource/')
    elif tagme and args.embedding_type == 'complex':
        tag = tag.replace('ENTITY/', '<http://dbpedia.org/resource/')
        tag = tag + '>'
    elif tag in redirect_dict:
        tag = redirect_dict[tag]
    else:
        tag = entity_converter(tag)
    if args.embedding_type == 'wikipedia2vec': 
        if tag in model.vocab:
            return model[tag]
    if args.embedding_type == 'rdf2vec': 
        if tag in model.vocab:
            return model[tag]
        else:
            return []
    if args.embedding_type == 'complex': 
        #if tag in model.dataset.entity_ids():
        if tag in entity_ids_dict:
            #ent_index = model.dataset.entity_ids().index(tag)
            ent_index = entity_ids_dict[tag]
            model_tag = model.get_s_embedder().embed(torch.Tensor([ent_index]).long())
            return model_tag
        else:
            return []
    else:
        return []


In [5]:
if args.embedding_type == 'complex':
    import torch
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
def ranking_feature(query, entity, dist = "cosine", conf = True):
    # Input: a query-entity pair to score, a distance function and if we want to use confidence scores
    # Output: query-entity based score

    # getting all the linked entities to the query
    #a_ent = pd.DataFrame(confidence.loc[confidence['query_id'] == query])
    if query not in confidence:
    #if len(a_ent)== 0:
    
        # if no linked queries, return 0
        return 0
    
    else:
        
# for interp in confidence['SemSearch_ES-6']['interpretations']:
#     for pr in interp['interpretation']:
#         print(pr)        
        
        max_total = []
        for a_ent in confidence[query]['interpretations']:
            total = 0
            ent_interps = [a for a in a_ent['interpretation'] if 'wikipedia' in a]
            ent_nr = 0

            for tag in ent_interps:
            #for index, row in a_ent.iterrows():
                #score = row['confidence']
                score = 1
                #score = row['score']
                if 'wikipedia' not in tag:
                    continue
                tag = 'ENTITY/' + tag.split('/')[-1]    
                ent1 = entity_lookup(entity, model)
                ent2 = entity_lookup(tag, model, tagme = True)
                #ent2 = entity_lookup(row['entity'])
                if len(ent1) == 0 or len(ent2) == 0:
                    continue
                else:
                    ent_nr += 1

                    if dist == 'fjaccard':
                        dist = 1-np.minimum(ent1,ent2).sum()/np.maximum(ent1,ent2).sum()
                    elif args.embedding_type == 'complex':
                        dist = cos(ent1, ent2).detach().numpy()[0]
                    else:
                        dist = 1 - spatial.distance.cosine(ent1, ent2)
                    if conf:
                        #score = row['confidence']
                        total += score*dist
                    else:
                        total += dist
            #total = total/ent_nr if total > 0 else 0
            #print(a_ent['interpretation'], total, ent_nr)
            max_total.append(total)
        #print(max_total, max(max_total))
        #returntotal = max(max_total)
        #returntotal = sum(max_total)/len(max_total)
        returntotal = sum(max_total)#/len(max_total)
        return returntotal

In [6]:
def to_print_format(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = new_df.loc[(new_df['query_id'] == q)]
        for index, row in entities.iterrows():
            printstring = str(int(row['rel'])) + ' qid:' + row['query_id'] + " 1:" + str(row['embedding_score']) + " 2:" + str(row['fsdm_score']) + " # " + row['tag']
            print(printstring, file = f)
    f.close()

In [7]:
def to_print_format_dict(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = rerank[q]
        for ent in entities.keys():
            printstring = str(int(entities[ent]['rel'])) + ' qid:' + q + " 1:" + str(entities[ent]['score']) + " 2:" + str(entities[ent]['fsdm_score']) + " # " + ent
            print(printstring, file = f)
    f.close()

In [9]:
print("Loading auxilary files")
# Loading the file to rerank
# Note, if an error is given here when loading a different file to rerank, try changing the seperator to '\t'



# qrels = pd.read_csv(qrels_path, sep='\t',names = ['query_id', '', 'tag', 'rel'])
# queries_path = path_to_dbpedia + '/collection/v2/queries-v2.txt'
# queries = pd.read_csv(queries_path, sep='\t',names = ['query_id', 'query'])

qrels_path = path_to_dbpedia + '/collection/v2/qrels-v2.txt'
qrels = {}
with open(qrels_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in qrels:
            qrels[terms[0]] = {}
        qrels[terms[0]][terms[2]] = terms[3]



rerank_path = args.dbpedia_input
#rerank =  pd.read_csv(rerank_path, sep='\s+', names = ['query_id', 'x1', 'tag', 'rang', 'fsdm_score', 'x2'])
rerank = {}
with open(rerank_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in rerank:
            rerank[terms[0]] = {}
        if terms[2] in qrels[terms[0]]:
            rel = qrels[terms[0]][terms[2]]
        else:
            rel = 0
        rerank[terms[0]][terms[2]] = {'fsdm_score' : terms[4], 'rel' : rel}



# Loading auxilary files
#qrels_path = path_to_dbpedia + '/collection/v2/qrels-v2.txt'
#qrels = pd.read_csv(qrels_path, sep='\t',names = ['query_id', '', 'tag', 'rel'])
queries_path = path_to_dbpedia + '/collection/v2/queries-v2.txt'
queries = pd.read_csv(queries_path, sep='\t',names = ['query_id', 'query'])

# Loading previously computed redirects
#df = pd.read_csv('/store/usr/gerritse/results_dict/wikipedia_redirect.csv')
df = pd.read_csv('../Data/wikipedia_redirect.csv')



Loading auxilary files


In [10]:
# Getting the data ready for further processing in RankLib
folds_path = path_to_dbpedia + "/collection/v2/folds/all_queries.json"
with open(folds_path, 'r') as read_file:
    data = json.load(read_file)

In [11]:
with open(args.linker) as json_file:
    ddata = json.load(json_file)
confidence = {}
for q in ddata['queries']:
    if 'dbpediav2' in q['id']:
        new_id = q['id'][12:]
        confidence[new_id] = q

In [12]:
len(confidence)

465

In [13]:
for interp in confidence['SemSearch_ES-6']['interpretations']:
    for pr in interp['interpretation']:
        print(pr)      

https://en.wikipedia.org/wiki/Air_Wisconsin
air
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(Cecil_Taylor_album)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(1971_album)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(Kym_album)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(Agua_de_Annique_album)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(band)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(singer)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(free_jazz_trio)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(2005_film)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(2015_film)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wiki/Air_(1977_video_game)
https://en.wikipedia.org/wiki/Wisconsin
https://en.wikipedia.org/wi

In [14]:
rerank['SemSearch_ES-6']

{'<dbpedia:Air_Wisconsin>': {'fsdm_score': '8.82690275', 'rel': '2'},
 '<dbpedia:Wisconsin_Air_National_Guard>': {'fsdm_score': '8.72861020',
  'rel': '0'},
 '<dbpedia:Air_Wisconsin_destinations>': {'fsdm_score': '8.57484073',
  'rel': '1'},
 '<dbpedia:Truax_Field_Air_National_Guard_Base>': {'fsdm_score': '8.47574436',
  'rel': '0'},
 '<dbpedia:1972_Lake_Winnebago_mid-air_collision>': {'fsdm_score': '8.38839447',
  'rel': '0'},
 '<dbpedia:Wisconsin_World_War_II_Army_Airfields>': {'fsdm_score': '8.31496520',
  'rel': '0'},
 '<dbpedia:128th_Air_Refueling_Wing>': {'fsdm_score': '8.18159120',
  'rel': '0'},
 '<dbpedia:Air_Wisconsin_Airlines>': {'fsdm_score': '8.15915830', 'rel': 0},
 '<dbpedia:126th_Air_Refueling_Squadron>': {'fsdm_score': '8.14427737',
  'rel': '0'},
 '<dbpedia:Volk_Field_Air_National_Guard_Base>': {'fsdm_score': '8.07942988',
  'rel': '0'},
 '<dbpedia:Dane_County_Regional_Airport>': {'fsdm_score': '8.04500137',
  'rel': '0'},
 '<dbpedia:General_Mitchell_Air_National_Guar

In [16]:
#confidence

In [15]:
print("Loading embeddings")
# Loading the model with a Gensim keyedvector
if args.embedding_type == 'wikipedia2vec' or args.embedding_type == 'rdf2vec': 
    model = gensim.models.KeyedVectors.load(args.embedding, mmap='r')
elif args.embedding_type == 'complex': 
    import torch
    from kge import Dataset
    from kge.model import KgeModel
    from kge.util.io import load_checkpoint
    checkpoint = load_checkpoint(args.embedding)
    dataset = Dataset.create(checkpoint['config'],folder = '../Data/dbpedia_kge_small')
    model = KgeModel.create_from(checkpoint, dataset)
    entity_ids = model.dataset.entity_ids()
    entity_ids_dict = {}
    for i, e in enumerate(entity_ids):
        entity_ids_dict[e] = i


redirect_dict = {}
for index, tags in df.iterrows():
    redirect_dict[tags['original']] = tags['redirect']

Loading embeddings


In [16]:
entity_lookup(entity_converter('<dbpedia:Amsterdam>'), model)

memmap([ 0.0267, -0.6669, -0.577 ,  0.1185,  0.3543,  0.3663,  0.6281,
        -0.6389,  0.0808, -0.2223,  0.3297, -0.1004, -0.9479,  0.4323,
         0.4797, -0.0581,  0.5623, -1.7071,  0.7628,  0.5647,  0.3995,
         0.7988, -0.3355,  0.2411, -0.9855, -0.8247, -0.6165, -1.2618,
        -0.7464,  0.084 , -0.5706, -0.1865, -0.4999, -0.4771,  0.6964,
        -0.3774,  0.1878,  0.019 , -0.5471,  1.0492,  0.2103, -0.7212,
         0.2763, -0.0965, -0.0764, -1.1733, -0.9302,  0.2558,  0.1496,
         0.5607, -0.2725, -0.0023,  0.6871,  0.4769, -0.6926, -0.779 ,
        -0.0532, -0.1834, -0.1065,  0.4034, -0.1091,  0.491 , -0.5935,
         0.4969, -0.284 ,  1.0307, -0.0463,  0.1828, -0.4415, -0.5461,
         0.4013, -0.2757,  0.1984, -0.914 ,  0.1912, -0.2991, -0.7957,
         1.1819,  0.2321, -0.2361, -0.6642, -0.1363,  0.538 , -0.6315,
        -0.4699,  0.207 , -0.0192, -0.0614,  0.4145,  0.8567, -0.4035,
         1.2693,  0.0799, -0.2084,  0.7934, -1.7709, -0.2036, -0.3543,
      

In [17]:
from timeit import default_timer as timer

start = timer()

ent1 = entity_lookup(entity_converter('<dbpedia:Nijmegen>'), model)
end = timer()
print(end - start)
ent2 = entity_lookup(entity_converter('<dbpedia:Amsterdam>'), model)
end = timer()
print(end - start)
end = timer()
print(end - start)
#spatial.distance.cosine(ent1, ent2)

9.182700887322426e-05
0.00041398778557777405
0.0005234419368207455


In [18]:
ranking_feature('SemSearch_ES-6',  '<dbpedia:Air_Wisconsin>', conf = True)

ranking_feature('SemSearch_ES-6',  '<dbpedia:Truax_Field_Air_National_Guard_Base>', conf = True)



11.446831032633781

In [22]:
# Merging the qrels and rerank for scoring
# new_df = pd.merge(qrels[['query_id','tag','rel']], rerank[['query_id','tag','fsdm_score']],  how='right', left_on=['query_id','tag'], right_on = ['query_id','tag'])
# new_df = new_df.fillna(value = 0)
# new_df['rel'].isna().sum()

# Scoring everything (might take a while)
test_x = []

f = open(args.score_output, 'w')
#N = len(new_df)
#l = 50
#m = (N/l)
start = timer()

print("Ranking entities:")
#for index, row in new_df.iterrows():
for query_id in rerank.keys():
    print(query_id)
    for tag in rerank[query_id].keys():        
        #i = int(index/m)
        #percentage = (100 * index) // N
        #print('[' + '-'*i + ' '*(l-1-i)+ '] ' + str(percentage) +"%", flush = True, end ='\r')
        #print(row['query_id'],row['tag'])
        query_based_score = ranking_feature(query_id, tag, conf = True)
        if query_based_score == 0:
            print(query_id, confidence[query_id])
            break
        test_x.append(query_based_score)
        #print(rerank[query_id][tag])
        rerank[query_id][tag]['score'] = query_based_score
        # start = timer()
        #print(" ".join([query_id, tag, str(query_based_score)] ), file=f)
        # #print(" ".join([row['query_id'], row['tag'], str(query_based_score)] ))
        # #if i > 1: 
        # #    break

    
# new_df['embedding_score'] = test_x
# print("\n")


Ranking entities:
INEX_LD-2012307
INEX_LD-2012307 {'id': 'esdbpediav2-INEX_LD-2012307', 'query': 'july 1850 president died millard fillmore sworn following day', 'difficulty': 3, 'categories': ['ConQ'], 'explicit_entities': [{'mention': 'millard fillmore', 'entity': ['https://en.wikipedia.org/wiki/Millard_Fillmore'], 'relevance': 2}], 'implicit_entities': [{'mention': 'july 1850 president', 'entity': ['https://en.wikipedia.org/wiki/Zachary_Taylor'], 'relevance': 2}], 'related_entities': [], 'interpretations': [{'id': 0, 'interpretation': ['july 1850', 'president', 'died', 'https://en.wikipedia.org/wiki/Millard_Fillmore', 'sworn', 'following day'], 'relevance': 3, 'equivalent': None, 'comment': None}, {'id': 1, 'interpretation': ['https://en.wikipedia.org/wiki/Zachary_Taylor', 'died', 'https://en.wikipedia.org/wiki/Millard_Fillmore', 'sworn', 'following day'], 'relevance': 3, 'equivalent': None, 'comment': None}]}
INEX_LD-2012321
INEX_LD-2012321 {'id': 'esdbpediav2-INEX_LD-2012321', 'qu

KeyError: 'INEX_XER-87'

In [32]:

print("Saving files for Ranklib:")
for i in range(5):
    query_fold = data[str(i)]
    folder = args.ranklib_output + "/Fold" +str(i+1)

    if not os.path.exists(folder):
        os.makedirs(folder)
    
    testpath = folder + "/test.txt"
    print("Now writing test for fold " + str(i+1))
    to_print_format_dict(query_fold['testing'], testpath)

    trainpath = folder + "/train.txt"
    print("Now writing train for fold " + str(i+1))
    test_print = to_print_format_dict(query_fold['training'], trainpath)


Saving files for Ranklib:
Now writing test for fold 1
Now writing train for fold 1
Now writing test for fold 2
Now writing train for fold 2
Now writing test for fold 3
Now writing train for fold 3
Now writing test for fold 4
Now writing train for fold 4
Now writing test for fold 5
Now writing train for fold 5
